In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame(columns=['prompts', 'answers'])

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from dotenv import load_dotenv
load_dotenv()
api_key=['OPENAI_API_KEY']

In [21]:
db = SQLDatabase.from_uri("sqlite:///../data/processed/test_prices.db")
llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')
cadena = SQLDatabaseChain(llm = llm, database = db, verbose=False)
formato = """
            Dada una pregunta del usuario:
            0. Convierte cualquier fecha al formato YYYY-MM-DD para mejor comprension del query
            1. crea una o varias consultas en las tablas de sqlite3 segun la pregunta (no le pongas limit a la consulta), si no se puede crear un query comunica que no tenemos data disponible de esos temas
            2. revisa los resultados, considera todas las columnas importantes al dar la respuesta
            3. devuelve el dato o los datos como un listado (incluye los valores de todas las columnas que consideres relevantes)
            4. añade un resumen con un analisis relevante de los datos mostrado y si hay noticias relacionadas en español, de no existir una noticia relevante, no la incluyas
            5. dale una estructura a la respuesta para que sea comoda de leer para el lector
            6. Si no sabes que responder, pregunta al usuario para obtener mas informacion, si no puedes obtener mas informacion, responde con el siguiente mensaje: "No tengo suficiente informacion para responder a tu pregunta"
            7. Siempre responde en español, y con palabras no tecnicas , un lenguaje cordial
            8. Siempre revisa la respuesta antes de enviarla, para asegurarte de que sea coherente y tenga sentido
            9. Siempre revisa la ortografia antes de enviar la respuesta
            10. Siempre que la respuesta contenga un dato numerico, añade la unidad de medida correspondiente y asegurate de que maximos tenga 2 decimales
            11. Siempre que la respuesta contenga la palabra null o none o NaN, cambia la respuesta a "No tenemos informacion disponible sobre ese tema"
            12. Si vas a presentar una tabla, asegurate de que los numeros maximo cuenten con 2 decimales.
            #{question}
            """

/home/juann/anaconda3/envs/BDI/lib/python3.11/site-packages/langchain_experimental/sql/base.py:76: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [22]:
def consulta(input_usuario):
    consulta = formato.format(question = input_usuario)
    
    try:
        resultado = cadena.run(consulta)
        return resultado
    
    except Exception as e:
        # If there's an exception, prompt ChatGPT for more information
        error_message = f"No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?"
        
        return error_message



In [7]:
#read file line by line
with open('generated_prompts.txt') as f:
    lines = f.readlines()
    for line in lines:
        answer = consulta(line)
        print(answer)
        with open('generated_answers_impact.txt', 'a') as f:
            f.write(answer)
            f.write('\n')

/home/juann/anaconda3/envs/BDI/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tengo suficiente información para responder a tu pregunta
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos información disponible sobre ese tema.
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos información disponible sobre ese tema.
No tenemos datos disponibles sobre ese tema. ¿Pu

In [23]:
import pandas as pd
# let's get the data from the file of prompts and answers
with open('generated_prompts.txt') as f:
    lines = f.readlines()
    for line in lines:
        answer = consulta(line)
        print(answer)
        new_row = {'prompts': line, 'answers': answer}
        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)


No tenemos información disponible sobre ese tema.
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos información disponible sobre ese tema.
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?
No tengo suficiente información para responder a tu pregunta.
No tenemos información disponible sobre ese tema.
No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionad

In [24]:
ans=df[df["answers"] != "No tenemos datos disponibles sobre ese tema. ¿Puede proporcionar más detalles o hacer otra pregunta relacionada?"]

In [25]:
# let check if any answer contains the word null
ans[ans['answers'].str.contains("null")]

,prompts,answers


In [26]:
#lets check if any answer contains the word None
ans[ans['answers'].str.contains("None")]

,prompts,answers


In [27]:
#lets check if any answer contains the word nan 
ans[ans['answers'].str.contains("nan")]

,prompts,answers


In [28]:
#lets check if any answer contains the word NaN
ans[ans['answers'].str.contains("NaN")]

,prompts,answers


In [29]:
#lets check if any answer contains the word Lo siento or have No tenemos
len(ans[ans['answers'].str.contains("Lo siento") | ans['answers'].str.contains("No tenemos")])

37

In [30]:
# lets iteratre over the answers
for index, row in ans.iterrows():
    print(row['prompts'])
    print(row['answers'])

¿Cuál ha sido el precio promedio del Maíz en León entre 2/2023 y 10/2023?

No tenemos información disponible sobre ese tema.
Listar los precios promedios mensuales de Maíz en León durante 2022.

No tengo suficiente información para responder a tu pregunta.
Identificar las provincias con los precios más altos para Maíz en 2022.

No tenemos información disponible sobre ese tema.
Identificar las provincias con los precios más altos para Maíz en 2022.

No tenemos información disponible sobre ese tema.
¿Cómo ha variado el precio del Maíz de variedad Internacional en los últimos meses?

El precio del Maíz de variedad Internacional en Madrid ha variado en los últimos meses de la siguiente manera:

- El precio el 1 de enero de 2021 fue de 220.19
- El precio el 1 de diciembre de 2020 fue de 202.68
- El precio el 1 de noviembre de 2020 fue de 211.84
- El precio el 1 de octubre de 2020 fue de 204.74
- El precio el 1 de septiembre de 2020 fue de 195.79

Análisis: El precio del Maíz de variedad Int

In [31]:
clean = ans[~ans['answers'].str.contains("Lo siento") & (~ans['answers'].str.contains("No tenemos"))  & (~ans['answers'].str.contains("no tenemos"))]


In [32]:
len(clean)

24

In [33]:
for index, row in clean.iterrows():
    print(row['prompts'])
    print(row['answers'])

Listar los precios promedios mensuales de Maíz en León durante 2022.

No tengo suficiente información para responder a tu pregunta.
¿Cómo ha variado el precio del Maíz de variedad Internacional en los últimos meses?

El precio del Maíz de variedad Internacional en Madrid ha variado en los últimos meses de la siguiente manera:

- El precio el 1 de enero de 2021 fue de 220.19
- El precio el 1 de diciembre de 2020 fue de 202.68
- El precio el 1 de noviembre de 2020 fue de 211.84
- El precio el 1 de octubre de 2020 fue de 204.74
- El precio el 1 de septiembre de 2020 fue de 195.79

Análisis: El precio del Maíz de variedad Internacional en Madrid ha experimentado fluctuaciones en los últimos meses, con un aumento significativo a principios de 2021 y variaciones a lo largo de 2020. Es importante monitorear estas tendencias para tomar decisiones informadas en el mercado agrícola.
¿Cómo ha variado el precio del Maíz de variedad Nacional en los últimos meses?

No tengo suficiente información pa

In [34]:
#lets save the clean data
clean.to_csv('clean_answers3.csv', index=False)